<a href="https://colab.research.google.com/github/sofials2002/SOFIA/blob/master/MRD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Randomization Design

### Double-sided Marketplace

[Bajari et al. (2023)](https://www.amazon.science/publications/experimental-design-in-marketplaces) discuss a business scenario involving a video streaming service where viewers watch content provided by various content creators. The company is interested in understanding the impact of increasing the number of advertisements shown before the content begins (referred to as Pre-Roll Ads or PRAs) on the time viewers spend watching the content.

The specific business problem is to assess the effect of doubling the number of PRAs (from one to two) on the engagement of viewers with content creators. The company needs to determine whether increasing the number of PRAs will reduce viewer engagement and how content creators might react to this change, particularly in terms of adjusting the number of ads shown during the content (Mid-Roll Ads or MRAs). You can refer to the paper for more details on MRAs and PRAs.

### Experimental Setup

The viewer-creator pair $(i, j)$ is the unit of observation in this experiment. The outcome of interest $Y_{ij}$ is the time spent by viewer $i$ watching content from creator $j$. We are interested in the effect of doubling the number of PRAs to all viewers. This is represented by
$$
\tau=\frac{1}{I J} \sum_{i=1}^I \sum_{j=1}^J\left(Y_{ij}(\mathbf{T}) - Y_{ij}(\mathbf{C})\right) ,
$$
where $Y_{ij}(\mathbf{T})$ is the potential time spent when every viewer-created pair is exposed to two PRAs, $Y_{ij}(\mathbf{C})$ is the potential time spent when every viewer-created pair is exposed to one PRA, and $I$ and $J$ are the number of viewers and content creators, respectively.

$$
\mathbf{Y} =
\begin{pmatrix}
Y_{1,1} & Y_{1,2} & Y_{1,3} & Y_{1,4} & Y_{1,5} & Y_{1,6} \\
Y_{2,1} & Y_{2,2} & Y_{2,3} & Y_{2,4} & Y_{2,5} & Y_{2,6} \\
Y_{3,1} & Y_{3,2} & Y_{3,3} & Y_{3,4} & Y_{3,5} & Y_{3,6} \\
Y_{4,1} & Y_{4,2} & Y_{4,3} & Y_{4,4} & Y_{4,5} & Y_{4,6} \\
Y_{5,1} & Y_{5,2} & Y_{5,3} & Y_{5,4} & Y_{5,5} & Y_{5,6} \\
Y_{6,1} & Y_{6,2} & Y_{6,3} & Y_{6,4} & Y_{6,5} & Y_{6,6} \\
Y_{7,1} & Y_{7,2} & Y_{7,3} & Y_{7,4} & Y_{7,5} & Y_{7,6} \\
Y_{8,1} & Y_{8,2} & Y_{8,3} & Y_{8,4} & Y_{8,5} & Y_{8,6} \\
Y_{9,1} & Y_{9,2} & Y_{9,3} & Y_{9,4} & Y_{9,5} & Y_{9,6} \\
Y_{10,1} & Y_{10,2} & Y_{10,3} & Y_{10,4} & Y_{10,5} & Y_{10,6} \\
\end{pmatrix}
$$


- *Direct Effects:* changes in outcome for a viewer-creator pair $(i, j)$ caused directly by a change in exposure for that pair, $W_{i j}$.
- *Spillover Effects:* changes in outcome for viewer-creator pair $(i, j)$ caused indirectly by the change in exposure for another pair $(i', j') \neq(i, j)$.

When full exposure $\mathbf{T}$ is implemented, meaning the new policy of showing two PRAs is applied to all viewer-creator pairs, both direct and spillover effects occur. With the direct effect, the viewers' experience is impacted by the additional PRAs, which is expected to lead to fewer streams being watched overall. This effect is captured by a "dispersion" parameter, $\delta > 0$. This triggers an indirect response from content creators: as a result of the viewers' dispersion, some content creators may choose to reduce their MRAs to lessen the impact of the increased PRAs. This response is captured by a "premium" parameter, $\alpha > 0$.

In [ ]:
import numpy as np

rng = np.random.default_rng(42)

# Define the number of buyers (I) and sellers (J)
I = 4000  # Number of buyers
J = 100   # Number of sellers

In [ ]:
# Generate the potential outcomes with one PRA Y_ij(C)

# Probability that the outcome is zero
pi = 0.1
B_0 = rng.binomial(1, pi, (I, J))

# Mean 300 minutes/month per active content
mean0, sd0 = (300, 50)
F_0 = rng.gamma(mean0**2/sd0**2, sd0**2/mean0, (I, J))

# Potential outcomes when nobody is treated
Y_C = B_0 * F_0

Y_C[:10, :10].round(0).astype(int)

In [ ]:
# Generate the potential outcomes with double PRA Y_ij(T)

# Dispersion: decrease in time viewers spend watching a stream when PRAs are increased
muD, sdD = (0.05, 0.1)
alphaD = ((1-muD)/sdD**2 - 1/muD) * muD**2
betaD = alphaD * (1/muD - 1)
Dispersion = rng.beta(alphaD, betaD, size=(I, J))  # Direct effect

# Premium: extra time viewers spend watching a stream when MRAs are reduced
muA, sdA = (0.02, 0.1)
alphaA = ((1-muA)/sdA**2 - 1/muA) * muA**2
betaA = alphaA * (1/muA - 1)
Premium = rng.beta(alphaA, betaA, size=(I, J))

# Potential outcomes when everyone is treated
Y_T = Y_C * (1 + Premium - Dispersion)

Y_T[:10, :10].round(0).astype(int)

In [ ]:
# Average treatment effect
ATE = (Y_T - Y_C).mean()
print(f"ATE: {ATE:.4f}")

## Viewer-Randomized Experiment

Randomize viewers into treatment and control groups regardless of the content they are watching:

$$
\mathbf{W} =
\begin{pmatrix}
\textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} \\
\textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} \\
\textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} \\
\textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} \\
\textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} \\
\textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} \\
\textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} \\
\textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} \\
\textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} & \textcolor{red}{T} \\
\textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} & \textcolor{blue}{C} \\
\end{pmatrix}
$$

- *Direct Effect:* viewers in the treatment group that are exposed to two PRAs will spend less time watching any content
- *Indirect Effect:* some content creators might strategically reduce MRAs because of the risk of losing viewers with two PRAs (negative externality), but viewers in the control group might engage more with these content creators that are reducing MRAs (positive externality)



In [ ]:
# Randomize viewer treatment assignment
pv = 0.5
Dv = rng.binomial(1, pv, size=I)

# Control group gets the premium
Yvr_C = Y_C[Dv==0] * (1 + Premium[Dv==0])

# Treatment group gets the premium and the dispersion
Yvr_T = Y_C[Dv==1] * (1 + Premium[Dv==1] - Dispersion[Dv==1])

# Average treatment effect
ATE_vr = Yvr_T.mean() - Yvr_C.mean()
print(f"ATE: {ATE_vr:.4f}")

The comparison between the treated and control viewers is overestimating (in absolute value) the effect of the treatment.